In [1]:
import pynq
import numpy as np
from tqdm import tqdm
import time
#from sklearn.metrics import average_precision_score, roc_auc_score

# 通过PYNQ将合成电路写入FPGA中
ol_loda = pynq.Overlay('./Dummy_full.bit')
mSCD = ol_loda.model_SCD_0
mM2M = ol_loda.Multi2Multi_0
mM2O = ol_loda.Multi2One_0

CONTROL_REGISTER = 0x0
mM2O.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0)
}

In [2]:
def _load_via_txt(path):
    X = np.loadtxt(path, delimiter=",")
    return X

def get_data(data_file):
    #data_path = os.path.join(data_base_path, data_file)
    data_path = data_file
    if ".mat" in data_file:
        from scipy.io import loadmat
        f = loadmat(data_path)
        X = f['X']
    else:
        X = _load_via_txt(data_path)
    return X

X_train = get_data("./deepsig.dat")  # Load Aryhytmia data.
print("X_train.shape = ",X_train.shape)

X_train.shape =  (2048, 2)


In [3]:
# 使用AXI DMA进行存储器传输
dma_i = ol_loda.axi_dma_0.sendchannel
dma_q = ol_loda.axi_dma_1.sendchannel
dma_v = ol_loda.axi_dma_2.recvchannel
dma_a = ol_loda.axi_dma_3.recvchannel
dma_f = ol_loda.axi_dma_4.recvchannel
# 处理大量数据
N = 2048 # X_train.shape[0]
Nout = 32768
NSamples = 100

buff_x = pynq.allocate(shape=(NSamples*N,), dtype=np.float16)
buff_y = pynq.allocate(shape=(NSamples*N,), dtype=np.float16)
buff_value = pynq.allocate(shape=(NSamples*Nout,), dtype=np.float16)
buff_al = pynq.allocate(shape=(NSamples*Nout,), dtype=np.float16)
buff_fl = pynq.allocate(shape=(NSamples*Nout,), dtype=np.float16)

# print("X_train[0:N] = ",X_train[0:N][:])


for j in range (NSamples):
    buff_x[j*N:(j+1)*N] = X_train[0:N,0]
    buff_y[j*N:(j+1)*N] = X_train[0:N,1]
    

In [4]:
start = time.time()
mSCD.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
mM2M.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0
mM2O.write(CONTROL_REGISTER, 0x81) # 0x81 will set bit 0

dma_i.transfer(buff_x)
dma_q.transfer(buff_y)
dma_v.transfer(buff_value)
dma_a.transfer(buff_al)
dma_f.transfer(buff_fl)
dma_i.wait()
dma_q.wait()
# dma_r.wait()
end = time.time()
mSCD.write(CONTROL_REGISTER,0x00) # stop
mM2M.write(CONTROL_REGISTER,0x00) # stop
mM2O.write(CONTROL_REGISTER,0x00) # stop
time_cost= end-start
print('=> Running on FPGA')
print(time_cost)
print()

=> Running on FPGA
0.01610589027404785

